# Going Subphraseless

The current method for isolating phrase heads ([here](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb)) requires strenuous and ineloquent processing of BHSA subphrase relations. The subphrases are not always consistently encoded and suffer from numerous exceptional cases. The result is that the method is rather convoluted and ineloquent.

This notebook will explore the possibility of disconnecting semantic head analysis from the ETCBC subphrase encoding. 

A "semantic" head is the primary content word of a phrase, following Croft's "Primary Information Bearing Unit":

> **The noun and the verb are the PRIMARY INFORMATION_BEARING UNITS (PIBUs) of the phrase and clause respectively. In common parlance, they are the content words. PIBUs have major informational content that functional elements such as articles and [auxiliaries] do not have. (Croft, *Radical Construction Grammar*, 2001, 258; see also Shead, *Radical Frame Semantics and Biblical Hebrew*, 104)**

> **A (semantic) head is the profile equivalent that is the primary information-bearing unit, that is, the most contentful item that most closely profiles the same kind of thing that the whole constituent profiles. (ibid., 259)**

Croft also provides an additional criterion to "profile equivalence":

> **If the criterion of profile equivalence produces two candidates for headhood, the less schematic meaning is the PIBU; that is, the PIBU is the one with the narrower extension, in the formal semantic sense of that term (ibid., 259)**

## Inquiry

Can we isolate semantic phrase heads in BHSA using only the phrase_atom and phrase limits? This question indeed means that we  take the phrase_atom/phrase boundaries for granted. Empirically, the validity of BHSA phrase boundaries needs to be tested. But for now, the exercise of isolating semantic phrase heads could be seen as the first step towards reproducible phrase boundaries.

## Basic Concepts

A semantic head will most often stand in a syntactically independent position. For Hebrew nominal phrases, that essentially means a word which is not precided by a construct, and which is semantically central (excluding attributive slots (e.g. H + noun + H + ATTRIBUTIVE) or an adjectival slots (e.g. noun + noun as in אישׁ טוב).

Quantifier expressions present unique cases, which may be syntactically independent but semantically secondary. These are expressed through specialized lexical items such as cardinal numbers and qualitative quantifiers (e.g.  "כל" and "חצי").

Another complication is the use of nouns as prepositional items. Such uses can be seen with words like פני "face" such as לפני "in front," and even words like ראשׁ as in ראשׁ החדשׁ "beginning of the month." 

Other expressions of quantity, quality, and function provide similar complexities. These cases have to be specified in advance.

### Ambiguity

Considerable ambiguity is present in several of cases:

**`A B and C`**<br>
Given A, B, C == nominal words. Is their relationship `A // B // C` or `A+B // C`. In other words: **what is the relationship of two adjacent nominal words given a list?** Is B a descriptor of A or is it an independent element? 

**`A of B and C`**<br>
Is it, `(A of B) // (C)` or `(A of (B // C)`

Or even:

**`A of B C and D`**<br>
This pattern combines elements from both ambiguous cases.

To address these ambiguities we will apply a battery of disambiguation attempts. Some of those attempts will draw from corpus data, i.e. do we ever see `B and C` with the conjunction explicitly elsewhere in the corpus? Or do we ever see a `A of C` excplicitly in the corpus? Accents may also play a role: do we see a conjunctive or disjunctive accent between `B C`? 

## Prerequisites

A number of pre-defined word sets are needed for processing quantification and ambiguous adjacency. These sets are made available in the form of `wsets`, a dictionary containing word sets that are calculated in to the `wordsets` directory of this repository. The following wordsets have been defined:

* nominals – a set of word nodes with parts of speech and participles that have the potential to function as nominalized elements. The selected parts of speech are quite permissive: `{'subs', 'nmpr', 'adjv', 'advb', 'prde', 'prps', 'prin', 'inrg'}`. Since parts of speech are not taken as universal linguistic categories but only summaries of language-specific word tendencies (cf. Croft, *Radical Construction Grammar*, 2001), we consider that almost any part of speech can be used in a nominal pattern (or construction). There are some upper limits to this assumption, though. For instance, we exclude cojunctions, articles, prepositions, and negators. 
* prepositions – a word set consisting of words with a part of speech category of `prep`, a lexical set (`ls`) feature of `ppre` ("potential preposition"), as well as a select group of nouns like פני "face" which have been processed for prepositionality. 
* quantifiers - consists of word nodes that are cardinal numbers or qualitative quantifiers such as כל.
* mword – mapping from a word to its phonological word group ("masoretic word"); joins words on maqqeph and ø space
* accent_type – a mapping from a word to its accent type: conjunctive or disjunctive
* conj_pairs – a dict of observed conjunction pairings of lexemes in the corpus: `A & B`
* cons_pairs – a dict of observed construct pairings of lexemes in the corpus: `A of B`
* mom – mapping from word node to its mother word node for a specified relationship: `mom[A]['coord'] = B`
* kid – opposite of mom; mapping from word to its children nodes for a relationship: `kid[A]['cons'] = B`

**Let's get started**. We load the necessary functions and BHSA data (straight from source).

In [1]:
import collections
import pickle
import random
import re
from IPython.display import display, HTML
from datetime import datetime
from pprint import pprint
from tf.app import use
wsets = pickle.load(open('wordsets/wsets.pickle', 'rb'))
A = use('bhsa', hoist=globals(), silent=True)

   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


### Wordsets

In [2]:
wsets.keys()

dict_keys(['noms', 'preps', 'quants', 'accent_type', 'mwords', 'conj_pairs', 'cons_pairs'])

In [3]:
list(wsets['cons_pairs'].keys())[:10]

['>JC/', 'KL/', 'BN/', 'TPF[', 'MZBX/', 'B>R/', '<T/', 'XRB/', 'RWX/', 'MLK/']

In [4]:
#wsets['conj_pairs']['>JC/']

# Machinery

We could use some machinery to do the hard work of looking in and around a node. In the older approach we used TF search templates. But these are not very efficient at scale, and they are always bound by the limits of the query language. I take another approach here: a class which contains all kinds of functions, wherein concise linguistic logic can be encoded and tested against a single word.

In [5]:
from wordsets.positions import Positions, Walker

## Positions class

The `Positions` class enables concise access to adjacent nodes within a given context. This allows for us to write algorithms with query-like efficiency with all of the power of Python. 

This class is instantiated on a word node and can provide contextual look-up data for a given word.

For example, given a phrase containing the following word nodes:

> (85213, **85214**, 85215)

Given that the bolded node, `8214` is our `source` word, we instantiate the class, feeding in the node, the "phrase" string (which is the context we want to search around), and an instance of Text-Fabric (`tf`):

In [6]:
            # node   context   TF instance
            #  |        |      |
P = Positions(85214, 'phrase', A).get

If we want to obtain the word adjacent one space forward, we simply ask `P` for `1`, which gives us the next word in the phrase.

In [28]:
P(1)

85215

If we try to ask for 2 words forward, we go beyond the bounds of the phrase. But `P` handles this by returning nothing:

In [8]:
P(2)

To look back one word, we simply give a negative value:

In [9]:
P(-1)

85213

Finally, `P` can be used to quickly call features on these words. For instance, in order to get the lexeme of the word in front of `8214`:

In [10]:
P(1,'lex')

'JHWH/'

And if we want to get a number of features, we can just add them with space-separation to the string. The result is a feature set:

In [11]:
P(1,'lex nu')

{'JHWH/', 'sg'}

`P` can also handle features on the source node itself by giving a positionality of `0`:

In [12]:
P(0, 'lex')

'<JN/'

## Walker class

`Walker` performs a similar function to `Positions`, except that it allows for skipped-spaces. `Walker` has two directions: `ahead` or `back`. A walk back will iterate backward from the node within a supplied context and return the first result that evaluates to `True` from a supplied function.

In [29]:
         # node  context   TF instance  
         #  |       |      |
W = Walker(85214, 'phrase', A)

`Walker` is demonstrated below with the same word. A simple `lambda` function is used to test for the lexeme. The first node ahead of the instanced node with a lexeme of "JHWH" is returned.

In [31]:
W.ahead(lambda w: F.lex.v(w) == 'JHWH/')

85215

An alternative demonstrates the `None` returned on the lack of a valid match.

In [33]:
W.ahead(lambda w: F.lex.v(w) == 'BOOGABOOGA')

# Need for Semantic Data

The accurate processing of word connections depends on fuller semantic data than BHSA provides. Future semantic data could be stored in a similar way to word sets (`wsets`). 

For example, in the two phrases

> (Exod 25:39) ככר זהב טהור <br>
> (2 Sam 24:24) בכסף שקלים חמשׁים

we see that זהב and כסף, despite being in two different positions with two different words indicates a kind of "composed of" semantic concept: "round gold" (i.e. round composed of gold) and "silver shekels" (shekels composed of silver). To process these kinds of links, we need a list of nouns that often function as "material." But this is only the beginning. Many other words will have specific semantic values that motivate their syntactic behavior. Such a scope lies outside the bounds of this author's current project on Hebrew time phrases.

## A Compromise: Time Phrases

Since constructing these semantic classes is vastly time consuming, I want to start with a smaller set of cases. I will instead focus on parsing connections within time phrases for now. This is because I am analyzing time phrases in my current ongoing PhD project. 

In [13]:
timephrases = [ph for ph in F.otype.s('phrase_atom') 
                   if F.function.v(L.u(ph, 'phrase')[0]) == 'Time'
                   and len(L.d(ph, 'word')) > 1
                   and F.language.v(L.d(ph, 'word')[0]) == 'Hebrew'
            ]

print(f'{len(timephrases)} phrases ready')

3766 phrases ready


## Search & Display Functions

The functions below allow for fast searching and displaying of queries using the `Positions` class. The searches rely on the `Grammar` class, described further below.

In [21]:
def prettyconds(rulesets):
    '''
    Iterate through an explain dict for a rela
    and print out all of checked conditions.
    '''
    
    for ruleset in rulesets:
        name, src, tgt = ruleset['name'], ruleset['src'], ruleset['tgt']
        print(name)
        print(f'{src} -> {tgt}')
        
        for cond, value in ruleset['cnd'].items():
            print('{:<30} {:>30}'.format(cond, str(value)))
        
        print()
        
def showmatch(match):
    
    '''
    Displays a match from a Grammar test.
    '''
    
    hit, conds = match['match'], match['conds']
    
    if not hit:
        print('NO MATCHES')
        print('-'*20)
        A.pretty(L.u(conds[0]['src'], 'phrase_atom')[0], extraFeatures='sp st', withNodes=True)
        prettyconds(conds)
        return None

    name, src, tgt = hit['name'], hit['src'], hit['tgt']

    phrase = L.u(src, 'phrase_atom')[0]

    highlights = {src:'pink',
                  tgt:'lightgreen'}

    A.pretty(phrase, withNodes=True, extraFeatures='sp st', highlights=highlights)
    prettyconds(conds)
    display(HTML('<hr>'))
        
def test_search(relastr, show=10, end=None, name='', phrases=None):
    '''
    Searches phrases with the specified relation 
    and prints out their descriptive explanation.
    '''
    
    start = datetime.now()
    print('beginning search')
    
    # build a convenient test set of words
    phrases = phrases or [ph for ph in F.otype.s('phrase_atom') 
                   if F.typ.v(ph) in {'NP', 'PP'}
                   and len(L.d(ph, 'word')) > 1
              ]
    words = [w for ph in phrases for w in L.d(ph, 'word')]
    
    # random shuffle to get good diversity of examples
    random.shuffle(words)
    
    # set up grammar
    G = Grammar(wsets, A)
    
    matches = []
    append = matches.append
    
    # iterate and find matches on words
    for i,w in enumerate(words):

        # update every 5000 iterations
        if i%5000 == 0:
            print(f'\t{len(matches)} found ({i}/{len(words)})')
        
        # run grammar search
        test = G.tests[relastr](w)
        
        # save results
        if test['match']:
            if not name:
                append(test)
            elif test['match']['name'] == name:
                append(test)
            
        # stop at end
        if len(matches) == end:
            break
        
        
    # display
    print('done at', datetime.now() - start)
    print(len(matches), 'matches found...')
    print('showing', end)
    
    for match in matches[:show]:
        showmatch(match)

## Grammar class

While `Positions` provides concise access to context, `Grammar` classes contain a series of functions which test a bunch of conditions. The conditions are formed by testing word `Positions`. An example of the `Grammar` class is provided below.

In [22]:
class Grammar:
    
    def __init__(self, wsets, tf):
        self.tf = tf
        self.wsets = wsets
        self.tests = {
            'defi': self.defi,
            'prep': self.prep,
            'cons': self.cons,
            'adjv': self.adjv,
            'quant': self.quant,
        }
        # cache results for recursive calls?
        self.cache = collections.defaultdict(lambda:collections.defaultdict())
        
    def getP(self, w):
        return Positions(w, 'phrase_atom', self.tf).get
    
    def test(self, condtuple):
        test = [ruleset for ruleset in condtuple
                    if all(ruleset['cnd'].values())
               ]
        
        # NB rules are written so as to default
        # on the last-most matching item
        # this allows for more complex situations
        # to default if true
        if test:
            return test[-1]
        else:
            return {}
        
    def everymatch(self, w):
        '''
        Runs analysis for all tests on a word.
        Returns as dict of test:result.
        '''
        results = {}
        
        for name, test in self.tests.items():
            res = test(w)
            if res['match']:
                results[name] = res['match']
                
        return results
        
    def defi(self, w):
        '''
        Matches a definite article with noun.
        '''
        
        params = {'match': {}, 'conds':None}
        
        if not w:
            return params
                
        P = self.getP(w)
        
        params['conds'] = (
        
        {
            'name': 'definite',
            'src': P(0),
            'tgt': P(1),
            'cnd': {
                
                'P(0,sp) == art':
                    P(0,'sp') == 'art',
                
                'bool(P(1))':
                    bool(P(1))
            }
            
        },
        )
        
        params['match'] = self.test(params['conds'])
        return params
    
    def prep(self, w):
        '''
        Matches a definite article with noun.
        '''
        
        params = {'match': {}, 'conds':None}
        
        if not w:
            return params
                
        P = self.getP(w)
        
        params['conds'] = (
        
        {
            'name': 'prep no article',
            'src': P(0),
            'tgt': P(1),
            'cnd': {
                
                'P(0,sp) == art':
                    P(0,'sp') == 'art',
                
                'bool(P(1))':
                    bool(P(1))
            }
            
        },
        )
        
        params['match'] = self.test(params['conds'])
        return params
        
    def cons(self, w):
        
        '''
        Queries for construct relations
        on a word.
        '''
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        params['conds'] = (

            {
                'name': 'cons',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {

                    'P(-1, st) == c': 
                        P(-1,'st') == 'c',

                    'P(-1) not in quants|preps': 
                        P(-1) not in quants|preps,
                }
            },

            {
                'name': 'cons',
                'src': P(0),
                'tgt': P(-2),
                'cnd': {

                    'P(-1,sp) == art': 
                        P(-1,'sp') == 'art',

                    'P(-2, st) == c': 
                        P(-2,'st') == 'c',

                    'P(-2) not in quants|preps': 
                        P(-2) not in quants|preps,
                }
            }
        )

        params['match'] = self.test(params['conds'])
        return params

    def adjv(self, w):
        
        '''
        Searches for adjective relas on a word.
        '''
                
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        # check for recursive adjective matches 
        a2match = self.adjv(P(-1))
        
        common = {
            
            'P(-1) in noms':
                P(-1) in noms,
            
            'P(-1, st) & {NA, a}': 
                P(-1,'st') in {'NA', 'a'},   
            
            'P(-1) not in {quants|preps}':
                P(-1) not in quants|preps,
        }
                
        params['conds'] = (
            
            {
                'name': 'adjv',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{

                    'P(0, sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},

                })
            },
            
            {
                'name': 'adjv adjv',
                'src': P(0),
                'tgt': a2match.get('tgt', None),
                'cnd': dict(common, **{
                    
                    'P(0,sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},
                    
                     'self.adjv(P(-1)) and target != P(0)':
                        bool(a2match['match']) and a2match.get('tgt', None) != P(0)
                })
            },
            
           {
                'name': 'adverb',
                'src': P(0),
                'tgt': P(1),
                'cnd': {

                    'P(0,sp) == advb':
                        P(0,'sp') == 'advb',

                    'P(1) in noms':
                        P(1) in noms,
                }
            },
            
            {
                'name': 'attrib pattern',
                'src': P(0),
                'tgt': P(-2),
                'cnd': {
                    
                    'P(0,sp) not in {prep, art, conj}':
                        P(0,'sp') not in {'prep', 'art', 'conj'},
                    
                    'P(0) not in preps':
                        P(0) not in preps,
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2) in noms':
                        P(-2) in noms,
                    
                    'P(-2) not in quants':
                        P(-2) not in quants,
                    
                    'P(-2,st) in {NA, a}':
                        P(-2,'st') in {'NA', 'a'},
                    
                    'P(-2,sp) != advb':
                        P(-2,'sp') != 'advb',
                    
                }
                
            }

        )

        params['match'] = self.test(params['conds'])
        return params
                
    def quant(self, w):
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, noms = wsets['quants'], wsets['noms']
        get_quanted = self.get_quantified(w)
        
        qqbackward = self.quant(P(-1))
        
        common = {
            'P(0) in quants':
                P(0) in quants,
        }
        
        params['conds'] = (
        
            {
                'name': 'quant forward',
                'src': P(0),
                'tgt': get_quanted,
                'cnd': dict(common, **{
                    
                    'bool(get_quanted)':
                        bool(get_quanted)
                })
            },
            
            {
                'name': 'quant backward',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{
                
                    'not bool(get_quanted)':
                        not bool(get_quanted),
                    
                    'P(-1,st) in {NA, a}':
                        P(-1,'st') in {'NA', 'a'},
                    
                    'P(-1) in noms':
                        P(-1) in noms,
                    
                    'P(-1) not in quants':
                        P(-1) not in quants
                    
                })
            },
            
            {
                'name': 'quant backward w/ article',
                'src': P(0),
                'tgt': P(-2),
                'cnd': dict(common, **{
                
                    'not bool(get_quanted)':
                        not bool(get_quanted),
                    
                    'P(-1, sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2,st) in {NA, a}':
                        P(-2,'st') in {'NA', 'a'},
                    
                    'P(-2) in noms':
                        P(-2) in noms,
                    
                    'P(-1) not in quants':
                        P(-2) not in quants
                    
                })
            },
        
            {
                'name': 'quant quant backward',
                'src': P(0),
                'tgt': qqbackward['match'].get('tgt', None),
                'cnd': dict(common, **{
                
                    'not bool(get_quanted)':
                        not bool(get_quanted),
                    
                    'bool(qqbackward[match])':
                        bool(qqbackward['match']),
                })
            },
            
        
        )
        
        params['match'] = self.test(params['conds'])
        return params
    
    def get_quantified(self, word):
        '''
        Locates the first non-quant nominal word
        ahead of a quantifier.
        '''

        if not word:
            return None
        
        quants, noms = self.wsets['quants'], self.wsets['noms']
        
        P = G.getP(word)

        conds = (
            P(0), 
            P(0) not in quants,
            P(0) in noms,
        )

        # check this word
        if all(conds):
            return word

        # move up one
        else:
            return self.get_quantified(P(1))

In [23]:
G = Grammar(wsets, A)

In [24]:
wsets['accent_type'][76403]

'disjunct'

In [25]:
showmatch(G.defi(690))

definite
690 -> 691
P(0,sp) == art                                           True
bool(P(1))                                               True



## Testing

In [30]:
#test_search('defi', name='', show=100, end=10, phrases=timephrases)

In [83]:
A.table(A.search('''

phrase function=Time
    word sp=art

''')[:10], condenseType='phrase', withNodes=True)

  0.82s 2596 results


n,p,phrase,word
1,Genesis 2:2,בַּיֹּ֣ום הַשְּׁבִיעִ֔י 651923,688
2,Genesis 2:2,בַּיֹּ֣ום הַשְּׁבִיעִ֔י 651923,הַ 690
3,Genesis 2:2,בַּיֹּ֣ום הַשְּׁבִיעִ֔י 651929,698
4,Genesis 2:2,בַּיֹּ֣ום הַשְּׁבִיעִ֔י 651929,הַ 700
5,Genesis 3:8,לְר֣וּחַ הַיֹּ֑ום 652298,הַ 1327
6,Genesis 4:14,הַיֹּ֗ום 652659,הַ 1883
7,Genesis 6:4,בַּיָּמִ֣ים הָהֵם֒ 653115,2667
8,Genesis 6:4,בַּיָּמִ֣ים הָהֵם֒ 653115,הָ 2669
9,Genesis 6:5,כָּל־הַיֹּֽום׃ 653142,הַ 2715
10,Genesis 7:10,לְשִׁבְעַ֣ת הַיָּמִ֑ים 653396,הַ 3218


## Map Relations on Timephrase Words

These relations will be used to calculate heads and export edges.

In [23]:
relas = {}

nomless = []

phrases = [ph for ph in F.otype.s('phrase_atom')
              if F.function.v(L.u(ph,'phrase')[0]) == 'Time'
              and F.typ.v(ph) not in {'CP'} 
              and F.language.v(L.d(ph,'word')[0]) == 'Hebrew'
          ]

# set up Grammar class
G = Grammar(wsets, A)

# time it
start = datetime.now()

print(f'{datetime.now()-start} beginning analysis...')

for i, phrase in enumerate(phrases):
    
    noms = [w for w in L.d(phrase, 'word') if w in wsets['noms']]
    
    # track cases without nominals
    if not noms:
        nomless.append(phrase)
        
    # analyze all known relas
    for w in L.d(phrase, 'word'):
        analysis = G.everymatch(w)
        relas[w] = analysis
        
    # report status
    if i % 500 == 0 and i:
        print(f'\t{datetime.now()-start}\tdone with iter {i}/{len(phrases)}')
        
print(f'{datetime.now()-start}\tCOMPLETE')

0:00:00.000034 beginning analysis...
	0:00:18.879881	done with iter 500/4486
	0:00:39.929214	done with iter 1000/4486
	0:00:57.947479	done with iter 1500/4486
	0:01:14.297314	done with iter 2000/4486
	0:01:33.585624	done with iter 2500/4486
	0:01:51.659994	done with iter 3000/4486
	0:02:08.804490	done with iter 3500/4486
	0:02:24.042792	done with iter 4000/4486
0:02:40.812403	COMPLETE


In [28]:
relas.get(212733, 'NONE')

{'adjv': {'name': 'attrib pattern',
  'src': 212733,
  'tgt': 212731,
  'cnd': {'P(0,sp) not in {prep, art, conj}': True,
   'P(0) not in preps': True,
   'P(-1,sp) == art': True,
   'P(-2) in noms': True,
   'P(-2) not in quants': True,
   'P(-2,st) in {NA, a}': True,
   'P(-2,sp) != advb': True}}}

In [34]:
shuff = [ph for ph in phrases 
             if ph not in nomless
            and len(L.d(ph, 'word')) > 5
        ]
random.shuffle(shuff)
    
for phrase in shuff[:100]:

    A.pretty(phrase, withNodes=True)
    
    for w in L.d(phrase , 'word'):


        
        print(T.text(w), w)

        for rela, match in relas.get(w, {}).items():

            print(f'\t** {rela} ** ')
            print('\t', match)
            print()

        print('-'*20)
        
    print()
    display(HTML('<hr>'))

גַּ֣ם  9182
--------------------
בַּ 9183
--------------------
 9184
--------------------
לַּ֧יְלָה  9185
--------------------
הַ 9186
--------------------
ה֛וּא  9187
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 9187, 'tgt': 9185, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------



בִּ 204496
--------------------
שְׁנַ֨ת  204497
--------------------
שְׁלֹשִׁ֤ים  204498
	** cons ** 
	 {'name': 'cons', 'src': 204498, 'tgt': 204497, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

	** quant ** 
	 {'name': 'quant forward', 'src': 204498, 'tgt': 204501, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וָ 204499
--------------------
תֵ֨שַׁע֙  204500
	** quant ** 
	 {'name': 'quant forward', 'src': 204500, 'tgt': 204501, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֔ה  204501
--------------------



בְּ 217900
--------------------
יֹ֥ום  217901
--------------------
נַחֲלָ֖ה  217902
	** cons ** 
	 {'name': 'cons', 'src': 217902, 'tgt': 217901, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

--------------------
וּ 217903
--------------------
כְאֵ֥ב  217904
--------------------
אָנֽוּשׁ׃ ס  217905
	** adjv ** 
	 {'name': 'adjv', 'src': 217905, 'tgt': 217904, 'cnd': {'not self.disj(P(0))[match]': True, 'P(1) in noms': True, 'P(-1, st) & {NA, a}': True, 'P(-1) not in {quants|preps}': True, 'P(0, sp) in {adjv, verb}': True}}

--------------------



בַּ 388950
--------------------
 388951
--------------------
שַּׁבָּ֖ת  388952
--------------------
וּ 388953
--------------------
בְ 388954
--------------------
יֹ֣ום  388955
--------------------
קֹ֑דֶשׁ  388956
	** cons ** 
	 {'name': 'cons', 'src': 388956, 'tgt': 388955, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

--------------------



בִּ 202671
--------------------
שְׁנַ֨ת  202672
--------------------
עֶשְׂרִ֧ים  202673
	** cons ** 
	 {'name': 'cons', 'src': 202673, 'tgt': 202672, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

	** quant ** 
	 {'name': 'quant forward', 'src': 202673, 'tgt': 202676, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וְ 202674
--------------------
שָׁלֹ֛שׁ  202675
	** quant ** 
	 {'name': 'quant forward', 'src': 202675, 'tgt': 202676, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֖ה  202676
--------------------



...

בֶּ 48202
--------------------
 48203
--------------------
חָרִ֥ישׁ  48204
--------------------
וּ 48205
--------------------
בַ 48206
--------------------
 48207
--------------------
קָּצִ֖יר  48208
--------------------



בְּ 284966
--------------------
עֶ֣צֶם׀  284967
--------------------
הַ 284968
--------------------
יֹּ֣ום  284969
--------------------
הַ 284970
--------------------
זֶּ֗ה  284971
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 284971, 'tgt': 284969, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------



לַ 410874
--------------------
 410875
--------------------
שַּׁבָּתֹות֙  410876
--------------------
וְ 410877
--------------------
לֶ֣ 410878
--------------------
 410879
--------------------
חֳדָשִׁ֔ים  410880
--------------------
וְ 410881
--------------------
לַ 410882
--------------------
 410883
--------------------
מֹּ֣ועֲדֹ֔ות  410884
--------------------



בַּ 309108
--------------------
 309109
--------------------
קַּ֥יִץ  309110
--------------------
וּ 309111
--------------------
בָ 309112
--------------------
 309113
--------------------
חֹ֖רֶף  309114
--------------------



בְּ 35588
--------------------
עֶ֨צֶם֙  35589
--------------------
הַ 35590
--------------------
יֹּ֣ום  35591
--------------------
הַ 35592
--------------------
זֶּ֔ה  35593
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 35593, 'tgt': 35591, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------



In [23]:
A.show(A.search('''

phrase function=Time
    word st=c lex#KL/|<YM/|>XRJT/ ls#card
    <: word lex=H
    <: word lex=JWM/

'''), end=20, condenseType='phrase', withNodes=True)

  2.24s 14 results


In [43]:
class PhraseGrammar:
    
    def __init__(self, ...):
    
    def disj(self, w):
        
        '''
        Queries for disjunct patterns, wherein A || B
        and B does not connect to A 
        A number of constructions are covered by this function:
            * lists - parallel items in a list
            * apposition - parallel items where B further specifies A
        '''
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        conjpairs = wsets['conj_pairs']
        atype = wsets['accent_type']
        mword = wsets['mwords']
        
        # conditions common to several parameters
        common = {
            
           'P(0) in noms':
                P(0) in noms,

            'not {P(0), P(-1)} & quants':
                not {P(0), P(-1)} & quants,

            'P(-1) in noms':
                P(-1) in noms,

            'P(-1, st) == a': 
                P(-1,'st') == 'a',
            
        }
        
        # build the conditions
        params['conds'] = (

            {
                'name': 'disjoint conjpair',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{
                                        
                    'P(-1,lex) in conjpairs[P(0,lex)]':
                        P(-1,'lex') in conjpairs[P(0,'lex')],
                })
            },
            
            {
                'name': 'disjoint accent',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{
                    
                    'P(-1) == disjunct and P(-1) not in mword':
                        atype.get(P(-1)) == 'disjunct' and P(-1) not in mword[w],
                })
            },
        )

        params['match'] = self.test(params['conds'])
        return params
        
    def par(self, w):
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, noms, preps = wsets['quants'], wsets['noms'], wsets['preps']
        get_quanted = self.get_quantified(w)
        atype = wsets['accent_type']
        
        # run checks for disjoint, const, adjv
        disj = self.disj(w)
        cons = self.cons(w)
        adjv = self.adjv(w)
        quant = self.quant(P(-1))
        
        params['conds'] = (
        
            {
                'name': 'adjacent disjoint',
                'src': P(0),
                'tgt': disj['match'].get('tgt', None),
                'cnd': {
                    'disj match':
                        bool(disj['match'])
                }
            },
            
            {
                'name': 'adjacent, not const|adjv|quant',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {
                    
                    'P(0) in noms':
                        P(0) in noms,
                    
                    'P(-1) in noms':
                        P(-1) in noms,
                    
                    'not cons match':
                        not bool(cons['match']),
                    
                    'P(-1) not quant match':
                        not bool(quant['match']),
                    
                    'P(-1) != adjv[match][target]':
                        P(-1) != adjv['match'].get('target', None),
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent',
                'src': P(0),
                'tgt': P(-2),
                'cnd':{
                    
                    'P(-1,sp) == conj':
                        P(-1,'sp') == 'conj',
                    
                    'P(-2) in noms':
                        P(-2) in noms,
                    
                    'P(-2) atype == conj':
                        atype.get(P(-2)) == 'conjunct',
                    
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (article)',
                'src': P(0),
                'tgt': P(-3),
                'cnd':{
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2,sp) == conj':
                        P(-2,'sp') == 'conj',
                    
                    'P(-2) in noms':
                        P(-3) in noms,
                    
                    'P(-2) atype == conj':
                        atype.get(P(-3)) == 'conjunct',
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (prep)',
                'src': P(0),
                'tgt': P(-3),
                'cnd':{
                    
                    'P(-1,sp) in preps':
                        P(-1) in preps,
                    
                    'P(-2,sp) == conj':
                        P(-2,'sp') == 'conj',
                    
                    'P(-3) in noms':
                        P(-3) in noms,
                    
                    'P(-3) atype == conj':
                        atype.get(P(-3)) == 'conjunct',
                    
                    'P(-4) in preps':
                        P(-4) in preps,
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (prep & article)',
                'src': P(0),
                'tgt': P(-4),
                'cnd':{
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2) in preps':
                        P(-2) in preps,
                    
                    'P(-3,sp) == conj':
                        P(-3,'sp') == 'conj',
                    
                    'P(-4) in noms':
                        P(-4) in noms,
                    
                    'P(-4) atype == conj':
                        atype.get(P(-4)) == 'conjunct',
                    
                    'P(-5,sp) == art':
                        P(-5,'sp') == 'art',
                    
                    'P(-6) in preps':
                        P(-6) in preps,
                    
                }
                
            },
        )
        
        params['match'] = self.test(params['conds'])
        return params